# 清洗104人力銀行職缺的"職缺名稱"資料

包含下列步驟 
> 字串清洗

> 計算職位名稱出現頻率，創建"職位名稱名單"與"排除名單"(Stopwords)。

> 清洗前的"職位名稱欄位"(jobName)與"職位名稱名單"對應，創建新欄位(jobName_clean)


#### 效果展示

    汐止分院護理人員    ----> 護理人員 
    汐止分院-專科護理師 ----> 專科護理師
    麵包製作人員        ----> 麵包製作人員
    理財專員           ----> 理財專員 
    天母店 餐廳正職人員 ----> 餐廳正職人員 

## 1. Read Job data from MongoDB

In [1]:
import pymongo
from secrets import HOST

建立MongoDB連線，並取出資料中的職缺名稱與分類

In [2]:
def mongo_connect_build(db_name, col_name): 
    
    global mycol
    client = pymongo.MongoClient(HOST)  # 跟 mongodb建立連線

    db = client[db_name] # 選擇使用的db
    mycol = db[col_name] # 選擇collection

In [3]:
# filter out job title data from JSON file

def extract_job_title(job_data):
    '''
    input: dictionary data from mongoDB
    output: list of data containing job title and category
    '''
    lst_jobTile = []

    for i in job_data:
        job_URL = list(i.keys())[1] # get the "URL shortcut" of each job
        job_content = i[job_URL]
        job_name = job_content['jobName']
        
        # get job category if column exist
        try:  
            jobCategory = job_content['jobCategory'][0]['description'] 
        except:
            jobCategory = None  

        dict_job = {'jobURL': job_URL, 'jobName': job_name, 'jobCategory': jobCategory }
        lst_jobTile.append(dict_job)
        
    return lst_jobTile

In [4]:
mongo_connect_build('Topic_104', 'Jobs') 
job_data = mycol.find() # 從mongoDB尋找多筆資料
lst_jobTitle = extract_job_title(job_data) # extract job title 

In [5]:
# 顯示資料清洗前職缺名稱
for i in lst_jobTitle[0:10]: 
    print(i)

{'jobURL': '1006w', 'jobName': '汐止分院護理人員(內外科病房.加護病房.門診.安寧病房.護理之家)', 'jobCategory': '護理師及護士'}
{'jobURL': '100ry', 'jobName': '汐止分院-專科護理師(內科.外科)', 'jobCategory': '護理師及護士'}
{'jobURL': '100w0', 'jobName': '麵包製作人員(正職)(台北南西店)', 'jobCategory': '麵包師'}
{'jobURL': '1020i', 'jobName': '理財專員(土城)(凱基證券)', 'jobCategory': '金融營業員'}
{'jobURL': '1034s', 'jobName': '天母店 餐廳正職人員', 'jobCategory': '餐飲服務生'}
{'jobURL': '1047q', 'jobName': '國內業務主任', 'jobCategory': '國內業務人員'}
{'jobURL': '106af', 'jobName': '正職行政人員(士林劍潭教學中心)', 'jobCategory': '補習班導師／管理人員'}
{'jobURL': '107xd', 'jobName': '系統工程師(技術服務部)', 'jobCategory': '網路管理工程師'}
{'jobURL': '10ccd', 'jobName': '[NB]軟體品保工程師(士林)', 'jobCategory': '品管／品保工程師'}
{'jobURL': '1n5g', 'jobName': '國內業務專員', 'jobCategory': '國內業務人員'}


## 2. Create Job Title list  (創建清洗後職缺名稱表單)

In [6]:
import re

### a.移除所有職缺名稱中，有括號的部分

In [7]:
for i in lst_jobTitle:
    i['jobName'] = re.sub(u"\\.|k|,|\+|\\(.*?\\)|\\{.*?}|\\[.*?]|\\【.*?】|\\（.*?）|\\(.*?）|\\<.*?>|\d|", "", i['jobName'])

for i in lst_jobTitle[0:10]:
    print(i)

{'jobURL': '1006w', 'jobName': '汐止分院護理人員', 'jobCategory': '護理師及護士'}
{'jobURL': '100ry', 'jobName': '汐止分院-專科護理師', 'jobCategory': '護理師及護士'}
{'jobURL': '100w0', 'jobName': '麵包製作人員', 'jobCategory': '麵包師'}
{'jobURL': '1020i', 'jobName': '理財專員', 'jobCategory': '金融營業員'}
{'jobURL': '1034s', 'jobName': '天母店 餐廳正職人員', 'jobCategory': '餐飲服務生'}
{'jobURL': '1047q', 'jobName': '國內業務主任', 'jobCategory': '國內業務人員'}
{'jobURL': '106af', 'jobName': '正職行政人員', 'jobCategory': '補習班導師／管理人員'}
{'jobURL': '107xd', 'jobName': '系統工程師', 'jobCategory': '網路管理工程師'}
{'jobURL': '10ccd', 'jobName': '軟體品保工程師', 'jobCategory': '品管／品保工程師'}
{'jobURL': '1n5g', 'jobName': '國內業務專員', 'jobCategory': '國內業務人員'}


### b. 計算"職缺名稱"出現頻率

In [8]:
def isEnglish(s): # 檢查字元是否為英文
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True


def count_job_CN(lst_jobTitle): # count the frequency of words in job title
    jobTitle_count = {}
    
    for i in lst_jobTitle:
        if isEnglish(i['jobName'])== False : # 處理非英文職缺
            JobName = re.sub(r'[^\w\s]',' ',i['jobName'])  # remove all punctuations
            JobName = JobName.replace('_', ' ')

            for word in JobName.split(' '): 
                word = word.strip()
                if len(word) > 1: 
                    if word not in jobTitle_count.keys():
                        jobTitle_count[word] = 1
                    else:
                        jobTitle_count[word] += 1
                        
    def sort_dict_by_value(x):  
        return {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}

    result = sort_dict_by_value(jobTitle_count)
    return result

In [9]:
count_result = count_job_CN(lst_jobTitle)  # 計算職位出現頻率

顯示20筆結果，結果顯示需要進一步清洗非職缺字元

In [10]:
for k,v in list(count_result.items())[0:20]: 
    print('職缺名稱: {} ， 出現頻率: {} '.format(k,v))

職缺名稱: 儲備幹部 ， 出現頻率: 2156 
職缺名稱: 業務助理 ， 出現頻率: 1550 
職缺名稱: 正職人員 ， 出現頻率: 1345 
職缺名稱: 業務專員 ， 出現頻率: 1280 
職缺名稱: 門市人員 ， 出現頻率: 1094 
職缺名稱: 業務人員 ， 出現頻率: 1044 
職缺名稱: 工程師 ， 出現頻率: 1024 
職缺名稱: 兼職 ， 出現頻率: 993 
職缺名稱: Engineer ， 出現頻率: 951 
職缺名稱: 行政助理 ， 出現頻率: 880 
職缺名稱: 工讀生 ， 出現頻率: 864 
職缺名稱: 兼職人員 ， 出現頻率: 864 
職缺名稱: 作業員 ， 出現頻率: 853 
職缺名稱: 台北 ， 出現頻率: 816 
職缺名稱: 軟體工程師 ， 出現頻率: 749 
職缺名稱: 無經驗可 ， 出現頻率: 671 
職缺名稱: 急徵 ， 出現頻率: 667 
職缺名稱: 計時人員 ， 出現頻率: 663 
職缺名稱: Manager ， 出現頻率: 633 
職缺名稱: 時薪 ， 出現頻率: 624 


### c. 創建職業名稱表

利用自行輸入職位表，跟排除表，篩選正確職缺

In [11]:
# 人工輸入職缺, 開啟'Jobtitle_HandMade.txt'檔案
with open(file='jieba_data/Jobtitle_HandMade.txt',mode='r', encoding="UTF-8") as file:
    handmade_words = file.read().split('\n')

# stopword 排除字元表單 ,開啟'Jobtitle_stopwords.txt'檔案
with open(file='jieba_data/Jobtitle_stopwords.txt',mode='r', encoding="UTF-8") as file:
    stop_words = file.read().split('\n')

In [12]:
# 定義創建職業名稱函數

def create_jobTitle(dict_jobCount, handmade_words):
    
    # 創建職業名稱list
    job_lst_clean = []
    
    # 排除詞彙 : 如果詞彙出現頻率 < 40, 詞彙長度 < 2, 或是詞彙屬於stopword
    for k, v in dict_jobCount.items(): 
        if (v < 40) or (len(k) < 2) or (k in stop_words): 
            continue
        else:
            job_lst_clean.append(k)

    # 加入人工輸入職缺
    for i in handmade_words: 
        if i.strip() not in job_lst_clean:
            job_lst_clean.append(i)
        
    # 更改中英職缺排序，中文職缺優先
    job_lst_clean_CN = []
    job_lst_clean_EN = []
    
    for job in job_lst_clean:
        if isEnglish(job) == True:
            job_lst_clean_EN.append(job)
        else:
            job_lst_clean_CN.append(job)
        

    # 根據職缺長度排序
    job_lst_clean_CN = sorted(job_lst_clean_CN, key = len, reverse = True)
    job_lst_clean_EN = sorted(job_lst_clean_EN, key = len, reverse = True)
    
    job_lst_clean_CN = job_lst_clean_CN + job_lst_clean_EN
    
    return job_lst_clean_CN

創建職業名稱名單list

In [13]:
job_lst_clean = create_jobTitle(count_result, handmade_words)  

print("總共產生職稱數量: " ,len(job_lst_clean))

總共產生職稱數量:  898


顯示部分創建後結果

In [14]:
for word in job_lst_clean[100:120]: 
    print('職缺名稱: ', word)

職缺名稱:  設備助理工程師
職缺名稱:  資訊軟體工程師
職缺名稱:  前端網頁工程師
職缺名稱:  生產技術工程師
職缺名稱:  技術服務工程師
職缺名稱:  製程助理工程師
職缺名稱:  硬體設計工程師
職缺名稱:  業務支援工程師
職缺名稱:  系統應用工程師
職缺名稱:  機構研發工程師
職缺名稱:  軟體助理工程師
職缺名稱:  產品設計工程師
職缺名稱:  系統整合工程師
職缺名稱:  資深機構工程師
職缺名稱:  Java工程師
職缺名稱:  電子研發工程師
職缺名稱:  IC佈局工程師
職缺名稱:  韌體設計工程師
職缺名稱:  系統維運工程師
職缺名稱:  機電繪圖工程師


## 3. Mapping Job title (將清洗過的職缺與原職缺做比對，產生新的欄位)

In [15]:
def add_jobtitle(lst_jobTitle, job_lst_clean):
    '''
    input: 
    lst_jobTitle -> 清洗前的職缺資料，格式為list of dictionary
    job_lst_clean -> 清洗後的職缺名單
    
    output:
    lst_jobTitle 加上清洗後的職缺欄位
    
    '''
    count = 0
    for i in lst_jobTitle:
        jobName = i['jobName']
        match_title = None 

        # match jobtitle with jobName
        for JOB in job_lst_clean:
            if re.search(rf"\s*{JOB}\s*", jobName, re.IGNORECASE):  
                match_title = JOB
                break

        # check whether match is successful. If not, use origianl job name.
        if match_title==None:
            match_title = jobName

        i['jobName_clean'] = match_title

取5筆職缺展示清洗前後效果

In [16]:
add_jobtitle(lst_jobTitle[0:5], job_lst_clean)

In [17]:
for i in lst_jobTitle[0:5]:
    print(i['jobName'], '---->', i['jobName_clean'])

汐止分院護理人員 ----> 護理人員
汐止分院-專科護理師 ----> 專科護理師
麵包製作人員 ----> 麵包製作人員
理財專員 ----> 理財專員
天母店 餐廳正職人員 ----> 餐廳正職人員
